In [1]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk, Toplevel
from PIL import Image, ImageTk
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import os
import fitz  # PyMuPDF
import threading
import time

# Global variable to hold the file name
file_name = ""
model_name = ""
start_time = 0  # Global variable to store start time

def select_csv():
    global file_name
    file_path = filedialog.askopenfilename(title="Select CSV File", filetypes=(("CSV Files", "*.csv"),))
    if file_path:
        file_name = os.path.basename(file_path)
        csv_label.config(text=f"Selected File: {file_name}")

def select_model():
    global model_name, start_time
    clear_previous_results()  # Clear previous results
    model = model_var.get()
    if model == 1:
        model_name = "Naïve Bayesian Network"
        notebook_file = 'Naive-bayes-network.ipynb'
        cpt_output_file = "CPT_output_naive.txt"
        pdf_files = ['naiveBayesNetwork.pdf']
    elif model == 2:
        model_name = "Tree-Augmented Naïve Bayesian Network"
        notebook_file = 'Tree-Augmented-naive-bayes-network.ipynb'
        cpt_output_file = "CPT_output_tree.txt"
        pdf_files = ['Tree-Augmented.pdf']
    elif model == 3:
        model_name = "Complex Bayesian Network"
        notebook_file = 'Complex-bayesian-network.ipynb'
        cpt_output_file = "CPT_output_complex.txt"
        pdf_files = ['complexBayesian.pdf']
    elif model == 4:
        model_name = "Naïve Bayes Deep Learning"
        notebook_file = 'Naive Bayes Deep Learning.ipynb'
        cpt_output_file = "CPT_output_naive_deep_learning.txt"
        pdf_files = ['deep_naive.pdf']
    elif model == 5:
        model_name = "Tree-Augmented Bayes Deep Learning"
        notebook_file = 'Tree-Augmented Bayes Deep Learning.ipynb'
        cpt_output_file = "CPT_output_tree_augmented_deep_learning.txt"
        pdf_files = ['deep_treeAugmented.pdf']
    elif model == 6:
        model_name = "Complex Bayes Deep Learning"
        notebook_file = 'Complex Bayes Deep Learning.ipynb'
        cpt_output_file = "CPT_output_complex_deep_learning.txt"
        pdf_files = ['deep_complex.pdf']
    else:
        messagebox.showerror("Error", "Please select a model.")
        return

    if file_name:
        start_time = time.time()  # Record the start time
        threading.Thread(target=process_file, args=(model_name, notebook_file, file_name, cpt_output_file, pdf_files)).start()
    else:
        messagebox.showerror("Error", "Please select a CSV file first.")

def process_file(model_name, notebook_file, file_name, cpt_output_file, pdf_files):
    progress_bar.start()
    
    try:
        # Load the notebook
        with open(notebook_file, 'r') as f:
            nb = nbformat.read(f, as_version=4)

        # Execute the notebook and capture output
        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
        ep.preprocess(nb, {'metadata': {'path': './'}})
        
        # Capture outputs and save to a file
        cpt_output = ""
        for cell in nb.cells:
            if cell.cell_type == 'code':
                if 'outputs' in cell:
                    for output in cell['outputs']:
                        if 'text' in output:
                            cpt_output += output['text']
        
        with open(cpt_output_file, "w") as text_file:
            text_file.write(cpt_output)

        # Display the PDFs
        display_pdfs(pdf_files, model_name)

        # Calculate the execution time
        end_time = time.time()
        execution_time = end_time - start_time

        # Display the execution time
        display_execution_time(execution_time)

        progress_bar.stop()
        messagebox.showinfo("Success", f"{model_name} has been processed successfully.\nCPT and other metrics are saved to {cpt_output_file}")
    except Exception as e:
        progress_bar.stop()
        messagebox.showerror("Error", f"An error occurred while processing: {str(e)}")

def display_pdfs(pdf_paths, model_name):
    for widget in plot_frame.winfo_children():
        widget.destroy()
    
    # Add Dependency Network label above the plot
    dependency_label = tk.Label(plot_frame, text=f"{model_name} Dependencies", font=("Helvetica", 12, "bold"), bg="#f0f0f0")
    dependency_label.pack(pady=10)

    execution_time_label = tk.Label(plot_frame, text="", font=("Helvetica", 12), bg="#f0f0f0")
    execution_time_label.pack(pady=10)

    for pdf_path in pdf_paths:
        pdf_document = fitz.open(pdf_path)
        page = pdf_document.load_page(0)
        pix = page.get_pixmap()

        # Convert to a PIL image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Resize the image to 90% of the original size
        img = img.resize((int(pix.width * 0.85), int(pix.height * 0.85)), Image.ANTIALIAS)

        # Convert to ImageTk format
        img_tk = ImageTk.PhotoImage(image=img)

        img_label = tk.Label(plot_frame, image=img_tk)
        img_label.image = img_tk  # Keep a reference
        img_label.pack(side=tk.LEFT, padx=10, pady=10, expand=True)

    display_execution_time.execution_time_label = execution_time_label

def display_execution_time(execution_time):
    display_execution_time.execution_time_label.config(text=f"Execution Time: {execution_time:.2f} seconds")

def clear_previous_results():
    for widget in plot_frame.winfo_children():
        widget.destroy()
    display_execution_time.execution_time_label = None

def open_covid_severity_window():
    severity_window = Toplevel(root)
    severity_window.title("Calculate Covid Severity")
    severity_window.geometry("1200x600")
    severity_window.state('zoomed')  # Maximize the window on startup
    severity_window.configure(bg="#f0f0f0")

    # Symptoms and their definitions
    symptoms = [
        ("HA", "Headache"), 
        ("SL", "Loss of smell"), 
        ("AL", "Loss of appetite"), 
        ("CO", "Cough"), 
        ("FE", "Fever"), 
        ("HO", "Hoarseness"), 
        ("ST", "Sore throat"), 
        ("CP", "Chest pain"), 
        ("FA", "Fatigue"), 
        ("CN", "Confusion"), 
        ("MP", "Muscle pain"), 
        ("SB", "Shortness of breath"), 
        ("DI", "Diarrhea"), 
        ("AA", "Abdominal pain")
    ]

    # Create a dictionary for symptom checkboxes
    symptom_vars = {abbr: tk.IntVar() for abbr, _ in symptoms}

    # Create a frame to center the content
    center_frame = tk.Frame(severity_window, bg="#f0f0f0")
    center_frame.pack(expand=True, padx=20, pady=20)

    # Add a title above the frames
    tk.Label(center_frame, text="Please select the symptoms", font=("Helvetica", 16, "bold"), bg="#f0f0f0").grid(row=0, columnspan=4, pady=(0, 20))

    # Create the table header with labels and checkboxes in the same frame
    for i, (abbr, definition) in enumerate(symptoms):
        frame = tk.Frame(center_frame, bg="#e0e0e0", bd=2, relief="groove", width=250, height=80, padx=10, pady=5)
        frame.grid(row=(i // 4) + 1, column=i % 4, padx=10, pady=10, sticky="nsew")
        frame.grid_propagate(False)  # Prevent frame from resizing to fit contents
        tk.Label(frame, text=definition, font=("Helvetica", 12), bg="#e0e0e0", wraplength=200).pack(pady=(5, 0))
        tk.Checkbutton(frame, variable=symptom_vars[abbr], bg="#e0e0e0").pack(pady=(0, 5))

    # Function to calculate severity based on selections
    def calculate_severity():
        selected_symptoms = {abbr for abbr, var in symptom_vars.items() if var.get() == 1}

        severity_levels = {
            6: {"‘severe level three; abdominal and respiratory’": {"HA", "SL", "AL", "CO", "FE", "ST", "CP", "FA", "CN", "MP", "SB", "DI", "AA"}},
            5: {"‘severe level two; confusion’": {"HA", "SL", "CO", "FE", "ST", "CP", "FA", "CN", "MP", "SB", "DI"}},
            4: {"‘severe level one; fatigue’": {"HA", "SL", "CO", "FE", "ST", "CP", "FA", "MP", "DI"}},
            3: {"‘gastrointestinal’": {"HA", "SL", "CO", "FE", "ST", "CP", "MP", "DI"}},
            2: {"‘flu-like’ with fever": {"HA", "SL", "CO", "FE", "ST", "CP", "MP"}},
            1: {"‘flu-like’ with no fever": {"HA", "SL", "CO", "ST", "CP", "MP"}}
        }

        for level, definitions in severity_levels.items():
            for definition, symptoms in definitions.items():
                if symptoms.issubset(selected_symptoms):
                    result_label.config(text=f"The patient's covid severity level is {definition} (Level {level})", font=("Helvetica", 14, "bold"))

                    return
        
        result_label.config(text="The patient's covid severity level does not match any predefined levels.")

    # Function to clear all checkboxes
    def clear_checkboxes():
        for var in symptom_vars.values():
            var.set(0)
        result_label.config(text="")

    # Add the "Calculate Severity" button
    calculate_button = tk.Button(center_frame, text="Calculate Severity", command=calculate_severity, font=("Helvetica", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
    calculate_button.grid(row=(len(symptoms) // 4) + 2, column=1, pady=20)

    # Add the "Clear" button
    clear_button = tk.Button(center_frame, text="Clear", command=clear_checkboxes, font=("Helvetica", 12), bg="#f44336", fg="white", padx=10, pady=5)
    clear_button.grid(row=(len(symptoms) // 4) + 2, column=2, pady=20)

    # Add a label to display the result
    result_label = tk.Label(center_frame, text="", font=("Helvetica", 12), bg="#f0f0f0")
    result_label.grid(row=(len(symptoms) // 4) + 3, columnspan=4, pady=10)

# Main application window setup (unchanged)
root = tk.Tk()
root.title("BayesCovid: Uncovering Hidden and Complex Relations of Pandemic Dynamics using an AI-driven System")
root.geometry("1200x600")
root.state('zoomed')  # Maximize the window on startup
root.configure(bg="#f0f0f0")

# Create a label
title_label = tk.Label(root, text="BayesCovid \n Uncovering Hidden and Complex Relations of Pandemic Dynamics using an AI-driven System", font=("Helvetica", 16, "bold"), bg="#f0f0f0")
title_label.pack(pady=(10, 20))  # Add space after the main title

# Create a frame to hold the image and select CSV button
image_csv_frame = tk.Frame(root, bg="#f0f0f0")
image_csv_frame.pack(pady=10)

# Load the image
image_path = "BayesianModelling.png"
image = Image.open(image_path)
image = image.resize((200, 100), Image.ANTIALIAS)
photo = ImageTk.PhotoImage(image)

# Add the image to the frame
image_label = tk.Label(image_csv_frame, image=photo, bg="#f0f0f0")
image_label.pack(side=tk.LEFT, padx=10)

# Create a button to select the CSV file and add it to the frame
select_csv_button = tk.Button(image_csv_frame, text="Select CSV", command=select_csv, font=("Helvetica", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
select_csv_button.pack(side=tk.LEFT, padx=10)

# Label to show selected CSV file and add it to the frame
csv_label = tk.Label(image_csv_frame, text="No file selected", font=("Helvetica", 10), bg="#f0f0f0")
csv_label.pack(side=tk.LEFT, padx=10)

# Create a frame for the models and severity buttons
models_severity_frame = tk.Frame(root, bg="#f0f0f0")
models_severity_frame.pack(pady=10)

# Create a variable to store the selected model
model_var = tk.IntVar(value=0)  # Initialize the variable with value 0

# Create a frame for Bayesian Models
frame_border = tk.Frame(models_severity_frame, bd=2, relief="groove", padx=10, pady=10, bg="#f0f0f0")
frame_border.pack(side=tk.LEFT, padx=10)

frame = tk.Frame(frame_border, bg="#f0f0f0")
frame.pack()

# Add a bold title inside the rectangle before the radio buttons
frame_title = tk.Label(frame, text="Bayesian Models", font=("Helvetica", 13, "bold"), bg="#f0f0f0")
frame_title.grid(row=0, columnspan=2, pady=(0, 10))

# Create radio buttons for model selection in two columns
radio_1 = tk.Radiobutton(frame, text="Naïve Bayesian Network", variable=model_var, value=1, font=("Helvetica", 12), bg="#f0f0f0")
radio_1.grid(row=1, column=0, sticky=tk.W, padx=20, pady=5)
radio_2 = tk.Radiobutton(frame, text="Tree-Augmented Naïve Bayesian Network", variable=model_var, value=2, font=("Helvetica", 12), bg="#f0f0f0")
radio_2.grid(row=2, column=0, sticky=tk.W, padx=20, pady=5)
radio_3 = tk.Radiobutton(frame, text="Complex Bayesian Network", variable=model_var, value=3, font=("Helvetica", 12), bg="#f0f0f0")
radio_3.grid(row=3, column=0, sticky=tk.W, padx=20, pady=5)
radio_4 = tk.Radiobutton(frame, text="Naïve Bayes Deep Learning", variable=model_var, value=4, font=("Helvetica", 12), bg="#f0f0f0")
radio_4.grid(row=1, column=1, sticky=tk.W, padx=20, pady=5)
radio_5 = tk.Radiobutton(frame, text="Tree-Augmented Bayes Deep Learning", variable=model_var, value=5, font=("Helvetica", 12), bg="#f0f0f0")
radio_5.grid(row=2, column=1, sticky=tk.W, padx=20, pady=5)
radio_6 = tk.Radiobutton(frame, text="Complex Bayes Deep Learning", variable=model_var, value=6, font=("Helvetica", 12), bg="#f0f0f0")
radio_6.grid(row=3, column=1, sticky=tk.W, padx=20, pady=5)

# Create a separate frame for the Covid Severity section
severity_frame_border = tk.Frame(models_severity_frame, bd=2, relief="groove", padx=10, pady=10, bg="#f0f0f0")
severity_frame_border.pack(side=tk.LEFT, padx=10)

severity_frame = tk.Frame(severity_frame_border, bg="#f0f0f0")
severity_frame.pack()

# Add a bold title for the new rectangle
frame_title2 = tk.Label(severity_frame, text="Decision Support System (DSS) for Calculating Covid Severity", font=("Helvetica", 13, "bold"), bg="#f0f0f0")
frame_title2.grid(row=0, columnspan=2, pady=(0, 10))

# Create a button to open the new window for Covid severity calculation
severity_button = tk.Button(severity_frame, text="Calculate Covid Severity", command=open_covid_severity_window, font=("Helvetica", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
severity_button.grid(row=1, columnspan=2, pady=10)

# Create a frame to hold the analyze button and progress bar
right_frame = tk.Frame(root, bg="#f0f0f0")
right_frame.pack(side=tk.RIGHT, padx=10, pady=10)

# Create a button to analyze the selected model
analyze_button = tk.Button(root, text="Analyse", command=select_model, font=("Helvetica", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
analyze_button.pack(pady=20)

# Create a progress bar
progress_bar = ttk.Progressbar(root, orient="horizontal", length=300, mode="indeterminate")
progress_bar.pack(pady=10)

# Create a frame for the plot and execution time
plot_execution_frame = tk.Frame(root, bg="#f0f0f0")
plot_execution_frame.pack(fill=tk.BOTH, expand=True)

execution_time_frame = tk.Frame(plot_execution_frame, bg="#f0f0f0")
execution_time_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)

plot_frame = tk.Frame(plot_execution_frame, bg="#f0f0f0")
plot_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# Run the application
root.mainloop()

